<center>

# $\textbf{Merge Files}$

<center>

### $\textbf{Code}$

In [9]:
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.appName('MergeFiles').master("local").enableHiveSupport().getOrCreate()
spark

In [ ]:
# Set paths to the Parquet files
file_paths = ["FilesParquet/Covid.parquet", "FilesParquet/GDP.parquet", "FilesParquet/Inflation.parquet",
              "FilesParquet/Migration.parquet", "FilesParquet/Population.parquet", "FilesParquet/Tax.parquet", "FilesParquet/Unemployment.parquet"]

# Read Parquet files into DataFrames
dfs = [spark.read.parquet(file_path) for file_path in file_paths]

# Merge or outer join the DataFrames
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.join(df, on="country", how="outer")

24/03/03 22:40:25 ERROR Executor: Exception in task 0.0 in stage 51.0 (TID 22)4]
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.copy(UnsafeRow.java:493)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage22.smj_consumeFullOuterJoinRow_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage22.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$2.hasNext(WholeStageCodegenExec.scala:779)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:168)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(

In [ ]:
#Storing this dataframe in parquet
merged_df.write.mode("overwrite").parquet("FinalOuterParquet")
spark.read.parquet("FilesParquet/Covid").show()
spark.stop()